## Semester 1 Project Submission

Please fill out:
* Student name: 
* Instructor name: 


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

In [1]:


import sqlite3
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats

conn = sqlite3.connect('zippedData/im.db')



# Dylan Orndorf-Ronk

In [22]:
!ls
movie_gross = pd.read_csv("zippedData/bom.movie_gross.csv.gz")
movie_gross.head(15)
# ROI = movie_gross['domestic_gross'] + movie_gross['foreign_gross']
# movie_gross['return_on_investment'] = movie_gross['domestic_gross' + 'foreign_gross']
# movie_gross['domestic_gross'].astype(int)
movie_gross_copy = movie_gross.copy()
movie_gross_copy['foreign_gross'] = movie_gross_copy['foreign_gross'].fillna(0)
# movie_gross_copy['foreign_gross'] = movie_gross_copy['foreign_gross'].astype(float)
movie_gross_copy.info()

CONTRIBUTING.md
LICENSE.md
README.md
awesome.gif
movie_data_erd.jpeg
student.ipynb
zippedData
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   3387 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


# Alex Bonczkiewicz

# Alberto Torres

# Travis McCloughan

# Jaemin Lee

In [3]:


q = """
SELECT name 
FROM sqlite_master 
WHERE type='table';
"""
pd.read_sql(q, conn)



,name


In [4]:


movie_basics = pd.read_sql("""
SELECT * 
FROM movie_basics;
""",conn)
movie_basics



DatabaseError: Execution failed on sql '
SELECT * 
FROM movie_basics;
': no such table: movie_basics

In [5]:


movie_ratings = pd.read_sql("""
SELECT * 
FROM movie_ratings;
""",conn)
movie_ratings



DatabaseError: Execution failed on sql '
SELECT * 
FROM movie_ratings;
': no such table: movie_ratings

In [ ]:


movies_df = pd.read_sql("""
SELECT *
FROM movie_akas
INNER JOIN
    movie_basics
ON  movie_akas.movie_id = movie_basics.movie_id
LEFT JOIN
    movie_ratings
ON  movie_akas.movie_id = movie_ratings.movie_id
""",conn)
movies_df



In [ ]:


movies_genre = pd.read_sql("""
SELECT genres, sum(numvotes) as num_votes, AVG(averagerating) as avg_rating
FROM movie_basics
INNER JOIN movie_ratings
    ON movie_basics.movie_id = movie_ratings.movie_id
GROUP BY genres
ORDER BY num_votes DESC, avg_rating DESC
LIMIT 20
""",conn)
movies_genre



In [ ]:


#movies_df.groupby(by="region").sum().reset_index()

movies_df.sort_values(['numvotes','averagerating'], ascending=False).groupby(by="region")



In [ ]:


persons = pd.read_sql("""
SELECT * 
FROM PERSONS;
""",conn)
persons



In [ ]:


movie_akas = pd.read_sql("""
SELECT * 
FROM movie_akas;
""",conn)
movie_akas



In [ ]:


directors = pd.read_sql("""
SELECT * 
FROM directors;
""",conn)
directors.head()



In [ ]:


principals = pd.read_sql("""
SELECT * 
FROM principals;
""",conn)
principals



In [ ]:


known_for = pd.read_sql("""
SELECT * 
FROM known_for;
""",conn)
known_for['movie_id'].value_counts()

